### General code

##### Import libraries and define custom functions

In [ ]:
# IMPORT LIRARIES
#----------------------------------------------------------------
# default libraries
import numpy as np
from random import sample
from time import time
import glob
import math
from functools import cache, wraps

# plotting libraries
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import host_subplot

# machine learning libraries
from keras.layers import Conv2D, Input, Dense, Dropout, MaxPool2D, UpSampling2D, Add
from keras.models import Model, Sequential
from keras.datasets import mnist, fashion_mnist, cifar10
from keras.callbacks import TensorBoard
from keras.utils import image_dataset_from_directory, split_dataset
import tensorflow as tf

# generic image functionality library
from skimage import data, img_as_float
from skimage.color import rgb2gray
from skimage.draw import disk
from sklearn.datasets import fetch_lfw_people

# image manipulation library
from scipy.fft import fft2, ifft2
from scipy.signal import fftconvolve as fftc
import cv2

# image deconvolution and metric library
from skimage.restoration import wiener, unsupervised_wiener, richardson_lucy
from skimage.metrics import mean_squared_error, peak_signal_noise_ratio, structural_similarity

# suppress warnings
""" import warnings
warnings.filterwarnings('ignore') """

print(f'Libraries have been loaded.')

In [ ]:
# DEFINE CUSTOM FUNCTIONS
#----------------------------------------------------------------
# timing functionality
def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        dur= te-ts
        print(f'func: {f.__name__} took: {dur:.3} sec')
        return result
    return wrap

# apply timing to imported functions
wiener_t = timing(wiener)
unsupervised_wiener_t = timing(unsupervised_wiener)
richardson_lucy_t = timing(richardson_lucy)

# presentation functions
def plot_images(images, titles=['undefined'], make_title=True, n_split=4, fname=[], scale=[0,1], colormap='gray'):
    # source: https://github.com/TristanvanLeeuwen/Summerschool
    """ helper function for plotting (originally K)
    Note: quite significantly modified for personal use """
    n = np.minimum(n_split, len(images))
    m = int(np.ceil(len(images)/n))
    img_array = [images[i * n:(i + 1) * n] for i in range((len(images) + n - 1) // n )]
    title_array = [titles[i * n:(i + 1) * n] for i in range((len(titles) + n - 1) // n )]
    fig, ax = plt.subplots(m, n, squeeze=False, figsize=(n*4,m*4))

    for row, k in enumerate(img_array):
        for col, l in enumerate(k):
            ax[row,col].set_xticks([])
            ax[row,col].set_yticks([])
            if scale == False:
                ax[row,col].imshow(img_array[row][col], cmap=colormap)
            else:
                ax[row,col].imshow(img_array[row][col], cmap=colormap, vmin=scale[0], vmax=scale[1])
            if make_title:
                try:
                    ax[row,col].set_title(title_array[row][col],fontsize=16)
                except IndexError:
                    ax[row,col].set_title(r'$undefined$',fontsize=16)
            
    list_c = [i for i in range(len(ax.flatten())) if i not in range(len(images))]
    for i in list_c:
        fig.delaxes(ax.flatten()[i])

    fig.tight_layout()
    
    if fname:
        plt.savefig(fname,dpi=300)

def single(image, size=None, osx=0, osy=0, colormap='gray', **kwargs):
    """ function to show a single image, or part of an image, without labels or axes """
    plt.figure(figsize=(4,4))
    plt.tick_params(left=False,
                    bottom=False,
                    labelleft=False,
                    labelbottom=False)
    if size is not None:
        plt.imshow(image[osx:osx+size,osy:osy+size], cmap=colormap, **kwargs)
    else:
        plt.imshow(image, cmap=colormap, **kwargs)
    plt.tight_layout()
    plt.show()

def print_quant(name, img1, img2, sig=3):
    """ print MSE, PSNR adn SSIM for two input images, with 'sig' number of significant numbers """
    img1 = normalize_image(img1)
    mse = mean_squared_error(img1,img2)
    psnr = peak_signal_noise_ratio(img1,img2)
    ssim = structural_similarity(img1,img2)
    print(f'Metrics for {name} are MSE: {mse:.3}, PSNR: {psnr:.3}, SSIM: {ssim:.3}')

def get_quant(img1, img2):
    img1 = normalize_image(img1)
    mse = mean_squared_error(img1,img2)
    psnr = peak_signal_noise_ratio(img1,img2)
    ssim = structural_similarity(img1,img2)
    cm = (-1*mse)+(psnr/30)+ssim
    return mse,psnr,ssim,cm

def plot_quants(params, quants, title=None, xl='parameter', cl=True, interval=1, legloc=7, cm=False, optweak=False, offset=True):
    # source: https://stackoverflow.com/a/45925049
    #----------------------------------------------------------------
    fig, host = plt.subplots(figsize=(8,5)) # (width, height) in inches
    # (see https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.subplots.html)
        
    par1 = host.twinx()
    par2 = host.twinx()
    if cm:
        par3 = host.twinx()

    host.set_ylabel("MSE")
    par1.set_ylabel("PSNR")
    par2.set_ylabel("SSIM")
    if cm:
        par3.set_ylabel("CM")
        par3.get_yaxis().set_visible(False)

    if title is not None:
        fig.suptitle(title, fontsize=12)
    if optweak:
        p1, = host.plot(params, quants[:,0], color='tab:green', label="MSE", alpha=0.5)
        p2, = par1.plot(params, quants[:,1], color='tab:blue', label="PSNR", alpha=0.5)
        p3, = par2.plot(params, quants[:,2], color='tab:orange', label="SSIM", alpha=0.5)
    else:
        p1, = host.plot(params, quants[:,0], color='tab:green', label="MSE")
        p2, = par1.plot(params, quants[:,1], color='tab:blue', label="PSNR")
        p3, = par2.plot(params, quants[:,2], color='tab:orange', label="SSIM")
    if cm:
        p4, = par3.plot(params, quants[:,3], color='tab:purple', label="CM", linestyle='dashed', marker='x')

    if cl:
        host.yaxis.label.set_color(p1.get_color())
        par1.yaxis.label.set_color(p2.get_color())
        par2.yaxis.label.set_color(p3.get_color())
        if cm:
            par3.yaxis.label.set_color(p4.get_color())

    lns = [p1, p2, p3]
    if cm:
        lns = [p1, p2, p3, p4]
    host.legend(handles=lns, loc=legloc)

    # plot optimum
    if offset:
        opti = get_optimum(quants, interval, offset=True)
    else:
        opti = get_optimum(quants, interval, offset=False)
    host.axvline(x=opti, color = 'tab:red', linestyle='dashed')

    host.set_xlabel(f'{xl}, optimal: {float(opti):.3}')

    # right, left, top, bottom
    par2.spines['right'].set_position(('outward', 60))

    #plt.show()
    plt.tight_layout()

def get_optimum(array, interval=1, offset=True):
    """ takes an array of MSE, PSNR and SSIM values and returns the index of the optimum values """
    """ mse_min = np.argmin(array[:,0])
    psnr_max = np.argmax(array[:,1])
    ssim_max = np.argmax(array[:,2])
    print(mse_min,psnr_max,ssim_max) """
    #array[:,0] = -1*array[:,0]
    #array[:,1] = array[:,1]/30
    #summa = np.sum(array,axis=1).tolist()
    #summa_max = np.argmax(summa)
    #result = summa_max*interval
    if offset:
        return (np.argmax(array[:,3])+1)*interval
    else:
        return np.argmax(array[:,3])*interval
    
# data structure modification
def batchify(input):
    return tf.expand_dims(input,0)

def blockshaped(arr, nrows, ncols):
    # source: https://stackoverflow.com/a/16858283
    """
    Return an array of shape (n, nrows, ncols) where
    n * nrows * ncols = arr.size

    If arr is a 2D array, the returned array should look like n subblocks with
    each subblock preserving the "physical" layout of arr.
    """
    h, w = arr.shape
    assert h % nrows == 0, f"{h} rows is not evenly divisible by {nrows}"
    assert w % ncols == 0, f"{w} cols is not evenly divisible by {ncols}"
    return (arr.reshape(h//nrows, nrows, -1, ncols).swapaxes(1,2).reshape(-1, nrows, ncols))
    
def into_chunks(img, dim_goal=28):
    """ function to cut up image into equally sized chunks with dimensions close to dim_goal """
    for i in range(dim_goal,((dim_goal*2)-1)):
        dim = img.shape[0]/i
        if dim.is_integer():
            # if the image is divisible by 'dim' then we can cut it up into chunks and write this to a variable
            print(f'Cutting up image into chunks of size: {int(i)}x{int(i)}')
            img_chunks = blockshaped(img, int(i), int(i))
            div_int = int(dim)
            break
    return img_chunks, div_int

def unblockinator(chunk_list, hor_n, ver_n):
    """ function to recombine a list of chunks made by 'into_chunks' """
    total_stack = []
    for i in range(ver_n):
        total_stack.append(np.hstack([x for x in chunk_list[i*hor_n:i*hor_n+hor_n]]))
    total_stack = np.vstack(total_stack)
    return total_stack

def get_kernel(full, kernel, mode='normal', crop_px=5):
    """ Get Kernel
    A simple function which takes the center part of any array given in the first argument based on the size of the second argument
    Allow the user to specify if they want the default behaviour, or a simple cropping function to crop the image by 'crop_px' pixels """
    if mode == 'normal':
        fullx, fully = full.shape
        kernelx, kernely = kernel.shape
        startx, endx = int(np.floor((fullx-kernelx)/2)), int(np.floor((fullx-kernelx)/2)+kernelx)
        starty, endy = int(np.floor((fully-kernely)/2)), int(np.floor((fully-kernelx)/2)+kernely)
        return full[startx:endx,starty:endy]
    elif mode == 'crop':
        fullx, fully = full.shape
        startx, endx = crop_px, fullx-crop_px
        starty, endy = crop_px, fully-crop_px
        return full[startx:endx,starty:endy]

def reshape(curr, goal, pad_val=0):
    """ this function take the psf and reshapes it to a size that matches that of the image """
    currx, curry = curr.shape
    goalx, goaly = goal.shape
    x_pad_affix = int(np.floor((goalx-currx)/2))
    x_pad_suffix = int(goalx-currx-x_pad_affix)
    y_pad_affix = int(np.floor((goaly-curry)/2))
    y_pad_suffix = int(goaly-curry-y_pad_affix)
    result = np.pad(curr, ((x_pad_affix, x_pad_suffix),(y_pad_affix, y_pad_suffix)), 'constant', constant_values=pad_val)
    return result

def disk_mask(img, fmod):
    #source: https://stackoverflow.com/a/70283438
    mask = np.zeros(img.shape, dtype=np.uint8)
    radius = fmod
    rr, cc = disk((0, 0), radius)
    mask[rr, cc] = 1
    return mask

# data modification
def normalize_image(img):
    """ this function normalize the image to the range [0,1] """
    img = img.astype(float)
    img = img - img.min()
    img = img / (img.max() - img.min())
    return img

def conv_prod(a, b, fmod=False):
    """ Convolution Product
    Takes two arguments; a and b, and returns the convolution product of the two """
    if a.shape[0] > b.shape[0]:
        if not fmod:
            return ifft2(fft2(a)*fft2(b,s=(a.shape[0],a.shape[1]))).real
        else:
            c = fft2(a)*fft2(b,s=(a.shape[0],a.shape[1]))
            # suppress frequencies radially starting at 'fmod'
            c = c*disk_mask(c,fmod)
            return ifft2(c).real
    elif a.shape[0] < b.shape[0]:
        return ifft2(fft2(b)*fft2(a,s=(b.shape[0],b.shape[1]))).real
    else:
        return ifft2(fft2(a)*fft2(b)).real
        
def normalize_complex_arr(a):
    # source: https://stackoverflow.com/a/41576956
    """ function to normalize complex arrays"""
    a_oo = a - a.real.min() - 1j*a.imag.min() # origin offsetted
    return a_oo/np.abs(a_oo).max()

def matlab_style_gauss2D(shape=(15,15),sigma=1):
    # source: https://stackoverflow.com/a/17201686
    """ 2D gaussian mask - should give the same result as MATLAB's fspecial('gaussian',[shape],[sigma]) """
    m,n = [(ss-1.)/2. for ss in shape]
    y,x = np.ogrid[-m:m+1,-n:n+1]
    h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
    h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
    sumh = h.sum()
    if sumh != 0:
        h /= sumh
    return h

# deconvolution schemes
# Naive
def decon_Naive(image, psf):
    """ Naive Deconvolution
    Takes two arguments; image and psf, and returns the deconvoluted result
    image (array): should be the distorted image
    psf (array): should be the point spread function """
    image_fourier = fft2(image)
    psf_fourier = fft2(psf,s=image.shape)
    return ifft2(image_fourier/psf_fourier).real

# Adjusted Wiener
def wiener_adj(image, psf, sigma=0.1):
    return ifft2(fft2(wiener(image, psf, sigma))/fft2(psf,s=image.shape)).real
    
# Richardson-Lucy
@timing
def RL_default(image, psf, niter, mode='normal', crop=5, quants=False, fmod=False):
    """ Richadson-Lucy Deconvolution
    Takes three arguments: image, psf, iter and returns the deconvoluted result
    image (array): should be the distorted image
    psf (array): should be the point spread function
    iter (integer): should be the number of Richadson-Lucy iterations to perform
    mode (string): 'normal' results in normal use of the RL algorithm, 'self' results in a blind self-iterative process """
    if quants:
        storage = []

    if mode == 'normal':
        base = image
        for x in range(niter):
            if not fmod:
                image = fftc(base/(fftc(image,psf,'same')+1e-12),np.flip(psf),'same')*image
            else:
                image = conv_prod(base/(conv_prod(image,psf,fmod=fmod)+1e-12),np.flip(psf),fmod=fmod)*image
            if quants:
                storage.append(get_quant(image,f))
    elif mode == 'crop':
        image = get_kernel(image, psf, mode='crop', crop_px=crop)
        f_crop = get_kernel(f, psf, mode='crop', crop_px=crop)
        base = image
        for x in range(niter):
            image = fftc(base/(fftc(image,psf,'same')+1e-12),np.flip(psf),'same')*image
            if quants:
                storage.append(get_quant(image,f_crop))
    elif mode == 'reflect':
        base_original = image
        base = np.pad(image, int(np.ceil(image.shape[0]*0.1)), mode='reflect')
        image = base
        for x in range(niter):
            if not fmod:
                image = fftc(base/(fftc(image,psf,'same')+1e-12),np.flip(psf),'same')*image
            else:
                image = conv_prod(base/(conv_prod(image,psf,fmod=fmod)+1e-12),np.flip(psf),fmod=fmod)*image
            if quants:
                result = get_kernel(image.real,base_original)
                storage.append(get_quant(result,f))
        image = get_kernel(image.real,base_original)
    elif mode == 'crofl':
        image = get_kernel(image, psf, mode='crop', crop_px=crop)
        f_crop = get_kernel(f, psf, mode='crop', crop_px=crop)
        base_original = image
        base = np.pad(image, int(np.ceil(image.shape[0]*0.1)), mode='reflect')
        image = base
        for x in range(niter):
            image = fftc(base/(fftc(image,psf,'same')+1e-12),np.flip(psf),'same')*image
            if quants:
                result = get_kernel(image.real,base_original)
                storage.append(get_quant(result,f_crop))
        image = get_kernel(image.real,base_original)
    if quants:
        storage = np.array(storage)
        return image.real, storage
    else:
        return image.real

# Richardson-Lucy using frequency suppression
def RL_default_gibbs(iters=1, mode='normal', fmod=50):
    result = RL_default(h,g,iters,mode=mode,fmod=fmod)
    return result

# Iterative Richardson-Lucy (iRL)
def RL_f_loop(f, g, c, nr_iter, clip):
    """ this function takes an initial f, g and c, and returns the new f after 'nr_iter' RL iterations """
    if g.shape != f.shape:
        # reshape and center the psf to match the size of the image to make convolution possible
        g = reshape(g, f)

    for i in range(nr_iter):
        #print('f',i)
        f = fftc(c/(fftc(f,g,'same')+1e-12),np.flip(g),'same')*f
        #f = normalize_complex_arr(f)
        #f /= np.sum(f)
    if clip:
        return np.clip(np.abs(f),0,1)
    else:
        return f

def RL_g_loop(f, g, c, nr_iter, clip):
    """ this function takes an initial f, g and c, and returns the new g after 'nr_iter' RL iterations """
    if g.shape != f.shape:
        # reshape and center the psf to match the size of the image to make convolution possible
        g = reshape(g, f)

    for i in range(nr_iter):
        #print('g',i)
        g = fftc(c/(fftc(g,f,'same')+1e-4),np.flip(f),'same')*g
        #g = normalize_complex_arr(g)
        #g /= np.sum(g)
    return g

@timing
def RL_iter(f, g, c, internal, nr_iter, mode='normal', clip_f=False, clip_g=False, inter_crop=True, quants=False):
    """ main Richardson-Lucy loop """
    i = 0
    f_base = f
    crop_kernel = np.zeros((15,15))
    if quants:
        storage = []
    while i < nr_iter:
        #print('loop',i)
        #print(np.sum(f),np.max(f))
        g = RL_g_loop(f, g, f_base, internal, clip_g)
        f = RL_f_loop(f, g, f_base, internal, clip_f)
        if inter_crop:
            g = get_kernel(g,crop_kernel)
        if quants:
            result = get_kernel(f.real,c)
            storage.append(get_quant(result,c))
        i += 1
    if quants:
        storage = np.array(storage)
        return f.real, g.real, storage
    else:
        return f.real, g.real

print(f'Custom functions have been loaded.')

First, let's take a look at what image convolution looks like.

In [ ]:
# LOAD DATA, MODIFY AND PRESENT
#----------------------------------------------------------------
# We can load an image for further use from the 'skimage' library
f = rgb2gray(data.astronaut()).astype('float32')

# Now we can create a PSF or convolution kernel (with sum(g) = 1)
g = np.ones((5,5))
g /= np.sum(g)

# Applying the convolution yields the following:
h = conv_prod(f,g)

# If we add a generic distortion in the form of random noise we get:
random_mask = 0.03 * np.random.randn(h.shape[0],h.shape[1])
f_noise = f + random_mask
f_noise = normalize_image(f_noise)
h_noise = h + random_mask
h_noise = normalize_image(h_noise)
# plot the images
plot_images([
        f, f_noise, h, h_noise
    ],[
        r'$f$', r'$f_{noise}$', r'$h$', r'$h_{noise}$'
    ])
# print quality materics for each image
print_quant('ground-truth',f,f)
print_quant('noisy image',f_noise,f)
print_quant('blurry image',h,f)
print_quant('noisy blurry image',h_noise,f)

### Analytical Methods

Now we can also start with some simple deconvolution methods.

In [ ]:
# NON-BLIND NAIVE DECONVOLUTION: NO NOISE, BLUR
#----------------------------------------------------------------
# compute the naive deconvolution
f_naive = decon_Naive(h,g)
# plot original, modified and reconstructed
plot_images([
        f, h, f_naive
    ],[
        r'$f$', r'$h$', r'$f_{naive}$'
    ])
# print similarity mertics
print_quant('naive deconvolution',f_naive,f)

In [ ]:
# NON-BLIND NAIVE DECONVOLUTION: NOISE, BLUR
#----------------------------------------------------------------
# compute the naive deconvolution
f_naive = decon_Naive(h_noise,g)
# plot original, modified and reconstructed
plot_images([
        f, h_noise, f_naive
    ],[
        r'$f$', r'$h_{noise}$', r'$f_{naive}$'
    ])
# print similarity mertics
print_quant('naive method applied to noisy blurry image',f_naive,f)

Instead of a naive deconvolution we can turn to the Wiener method to combat noise artifacts.

In [ ]:
# SEMI-BLIND DECONVOLUTION: WIENER
#----------------------------------------------------------------
# computed the Wiener deconvolution
wiener_01 = wiener(h_noise, g, 0.1)
wiener_02 = wiener(h_noise, g, 0.2)
wiener_05 = wiener(h_noise, g, 0.5)
# plot original, modified and reconstructed
plot_images([
        f, h_noise, wiener_01
    ],[
        r'$f$', r'$h_{noise}$', r'$f_{Wiener~0.1}$'
    ])
# print similarity mertics
print_quant('Wiener deconvolution (0.1)',wiener_01,f)
print_quant('Wiener deconvolution (0.2)',wiener_02,f)
print_quant('Wiener deconvolution (0.5)',wiener_05,f)

In [ ]:
# SEMI-BLIND DECONVOLUTION: WIENER --> GRAPH
#----------------------------------------------------------------
def graph_wiener(params=np.arange(0.1, 50, 0.5)):
    # create a list to store the results in
    storage = []
    # append quant results to empty list
    for i in params:
        result = wiener(h_noise, g, i)
        storage.append(get_quant(result,f))
    # transform list to numpy array
    storage = np.array(storage)
    # call the plot function for the list of parameters and the quant array
    plot_quants(params, storage, title='Metrics of Wiener per sigma', xl='Lambda', interval=0.5, cm=True, optweak=True) # interval = 'params' arange interval

graph_wiener()

In [ ]:
# SEMI-BLIND DECONVOLUTION: ADJUSTED WIENER
#----------------------------------------------------------------
# computed the default and adjusted Wiener deconvolutions
f_wiener_default = wiener(h_noise, g, 0.1)
f_wiener_adjusted = wiener_adj(h_noise, g, 0.1)
# plot original, modified and reconstructed
plot_images([
        h_noise, f_wiener_default, f_wiener_adjusted
    ],[
        r'$h_{noise}$', r'$f_{Wiener~3}$', r'$f_{Wiener~adjusted~3}$'
    ])
# print similarity mertics
print_quant('default Wiener method',f_wiener_default,f)
print_quant('adjusted Wiener method',f_wiener_adjusted,f)

In [ ]:
# SEMI-BLIND DECONVOLUTION: ADJUSTED WIENER --> GRAPH
#----------------------------------------------------------------
def graph_wiener_adj(params=np.arange(0.1, 50, 0.5)):
    # create a list to store the results in
    storage = []
    # append quant results to empty list
    for i in params:
        result = wiener_adj(h_noise, g, i)
        storage.append(get_quant(result,f))
    # transform list to numpy array
    storage = np.array(storage)
    # call the plot function for the list of parameters and the quant array
    plot_quants(params, storage, title='Metrics of adjusted Wiener per sigma', xl='Lambda', interval=0.5, cm=True, optweak=True) # interval = 'params' arange interval

graph_wiener_adj()

We can also account in part for uncertainty in our input parameters by using the unsupervised Wiener method.

In [ ]:
# SEMI-BLIND DECONVOLUTION: UNSUPERVISED WIENER
#----------------------------------------------------------------
# calculate unsupervised wiener deconvolutions for different sigma valued gaussian psf's
unsup_s1 = unsupervised_wiener(h_noise,matlab_style_gauss2D(sigma=1))[0]
unsup_s2 = unsupervised_wiener(h_noise,matlab_style_gauss2D(sigma=2))[0]
unsup_s3 = unsupervised_wiener(h_noise,matlab_style_gauss2D(sigma=3))[0]
# plot the results
plot_images([
        unsup_s1, unsup_s2, unsup_s3
    ],[
        r'$\sigma=1$', r'$\sigma=2$', r'$\sigma=3$'
    ])
# print similarity mertics
print_quant('unsupervised Wiener method with sigma=1',unsup_s1,f)
print_quant('unsupervised Wiener method with sigma=2',unsup_s2,f)
print_quant('unsupervised Wiener method with sigma=3',unsup_s3,f)

In [ ]:
# SEMI-BLIND DECONVOLUTION: UNSUPERVISED WIENER --> GRAPH
#----------------------------------------------------------------
def graph_wiener_unsup(params=np.arange(0.1, 5, 0.1)):
    # create a list to store the results in
    storage = []
    # append quant results to empty list
    for i in params:
        result = unsupervised_wiener(h_noise,matlab_style_gauss2D(sigma=i))[0]
        storage.append(get_quant(result,f))
    # transform list to numpy array
    storage = np.array(storage)
    # call the plot function for the list of parameters and the quant array
    plot_quants(params, storage, title='Metrics of unsupervised Wiener per sigma', xl='Sigma', interval=0.1, cm=True, optweak=True) # interval = 'params' arange interval

graph_wiener_unsup()

To improve the results we can try the blind/semi-blind Richardson-Lucy deconvolution method.

In [ ]:
# BLIND DECONVOLUTION: DEFAULT RICHARDSON-LUCY (NO NOISE, BLUR)
#----------------------------------------------------------------
# NOTE: the psf can be chosen to be either the same as the 'real' blur kernel, or a general gaussian.
#       uncomment only whichever one you would like to use
#_____ real blur kernel
#g = np.ones((5,5))
#g /= np.sum(g)
#_____ gaussian
g = matlab_style_gauss2D(shape=(15,15), sigma=2)
def rl_gallery(noise=False, noise_level=0.1):
    # calculate the default and custom RL deconvolutions at different numbers of iterations
    if not noise:
        rld_5 = richardson_lucy_t(h,g,5)
        rld_100 = richardson_lucy_t(h,g,100)
        rld_250 = richardson_lucy_t(h,g,250)
        rlc_5 = RL_default(h,g,5)
        rlc_100 = RL_default(h,g,100)
        rlc_250 = RL_default(h,g,250)
    else:
        random_mask = noise_level * np.random.randn(h.shape[0],h.shape[1])
        h_noise = h + random_mask
        h_noise = normalize_image(h_noise)
        rld_5 = richardson_lucy_t(h_noise,g,5)
        rld_100 = richardson_lucy_t(h_noise,g,100)
        rld_250 = richardson_lucy_t(h_noise,g,250)
        rlc_5 = RL_default(h_noise,g,5)
        rlc_100 = RL_default(h_noise,g,100)
        rlc_250 = RL_default(h_noise,g,250)
    # plot
    plot_images([
            f, rld_5, rld_100, rld_250, h, rlc_5, rlc_100, rlc_250
        ],[
            r'$f$', r'$RL_{5}$', r'$RL_{100}$', r'$RL_{250}$',
            r'$h$', r'$RLc_{5}$', r'$RLc_{100}$', r'$RLc_{250}$'
        ])
    # print similarity mertics
    print_quant('default RL at 5 iterations',rld_5,f)
    print_quant('default RL at 100 iterations',rld_100,f)
    print_quant('default RL at 250 iterations',rld_250,f)
    print_quant('custom RL at 5 iterations',rlc_5,f)
    print_quant('custom RL at 100 iterations',rlc_100,f)
    print_quant('custom RL at 250 iterations',rlc_250,f)

rl_gallery(noise=False)

In [ ]:
# COMPARISON SPEED scikit RL AND CUSTOM RL FUNCTIONS
#----------------------------------------------------------------
#single(richardson_lucy_t(h_noise,g,250))
#RL_default(h_noise,g,250)

In [ ]:
# PLOT HIGHLIGHTS (CHOOSE ONE): DEFAULT RICHARDSON-LUCY (NO NOISE, BLUR)
#----------------------------------------------------------------
# uncomment only the selection which you want to see
#single(rld_250, 100, 0, 0)
#single(rlc_250, 100, 0, 0)
#single(rld_250, 100, 100, 100)
#single(rlc_250, 100, 100, 100)

In [ ]:
# SEMI-BLIND DECONVOLUTION: RICHARDSON-LUCY DEFAULT --> GRAPH
#----------------------------------------------------------------
def graph_rl_defult(params=250, noise=False):
    # create a list to store the results in
    if not noise:
        result = RL_default(h,g,params,mode='normal',quants=True)[1]
    else:
        result = RL_default(h_noise,g,params,mode='normal',quants=True)[1]
    # call the plot function for the list of parameters and the quant array
    plot_quants(np.arange(0, params, 1), result, title=None, xl='Iterations', cm=False, optweak=False, offset=False)

graph_rl_defult(noise=False)

To minimize artifacting in the result image modifications can be applied and the default RL method can be modified, starting with reflective padding.

In [ ]:
# BLIND DECONVOLUTION: REFLECTIVE PADDING
#----------------------------------------------------------------
# produce a reflected exmaple of the blurred image
h_refl = np.pad(h, int(np.ceil(h.shape[0]*0.1)), mode='reflect')
def rl_refl_gallery(noise=False, noise_level=0.1):
    # calculate the reflected custom RL deconvolutions at different numbers of iterations
    if not noise:
        rlc_5_r = RL_default(h,g,5,mode='reflect')
        rlc_100_r = RL_default(h,g,100,mode='reflect')
        rlc_250_r = RL_default(h,g,250,mode='reflect')
        # plot
        plot_images([
                h, rlc_5_r, rlc_100_r, rlc_250_r
            ],[
                r'$h$', r'$f_{RL~reflect~5}$', r'$f_{RL~reflect~100}$', r'$f_{RL~reflect~250}$'
            ])
    else:
        random_mask = noise_level * np.random.randn(h.shape[0],h.shape[1])
        h_noise = h + random_mask
        h_noise = normalize_image(h_noise)
        rlc_5_r = RL_default(h_noise,g,5,mode='reflect')
        rlc_100_r = RL_default(h_noise,g,100,mode='reflect')
        rlc_250_r = RL_default(h_noise,g,250,mode='reflect')
        # plot
        plot_images([
                h_noise, rlc_5_r, rlc_100_r, rlc_250_r
            ],[
                r'$h_{noise}$', r'$f_{RL~reflect~5}$', r'$f_{RL~reflect~100}$', r'$f_{RL~reflect~250}$'
            ])
    # print similarity mertics
    print_quant('reflective RL at 5 iterations',rlc_5_r,f)
    print_quant('reflective RL at 100 iterations',rlc_100_r,f)
    print_quant('reflective RL at 250 iterations',rlc_250_r,f)

rl_refl_gallery(noise=False)

In [ ]:
# PLOT HIGHLIGHTS (CHOOSE ONE): REFLECTIVE RICHARDSON-LUCY (NO NOISE, BLUR)
#----------------------------------------------------------------
# uncomment only the selection which you want to see
#single(rlc_5_r, 100, 0, 0)
#single(rlc_100_r, 100, 0, 0)
#single(rlc_250_r, 100, 0, 0)

In [ ]:
# SEMI-BLIND DECONVOLUTION: REFLECTIVE RL --> GRAPH
#----------------------------------------------------------------
def graph_rl_refl(params=250):
    # create a list to store the results in
    result = RL_default(h,g,params,mode='reflect',quants=True)[1]
    # call the plot function for the list of parameters and the quant array
    plot_quants(np.arange(0, params, 1), result, title='Metrics of reflective RL over iterations', xl='Iterations')

graph_rl_refl()

Now to further improve a crop can be applied.

In [ ]:
# BLIND DECONVOLUTION: CROPPED
#----------------------------------------------------------------
def rl_crop_gallery(noise=False, noise_level=0.1):
    # calculate the cropped custom RL deconvolutions at different numbers of iterations
    if not noise:
        rlc_5_c = RL_default(h,g,5,mode='crop')
        rlc_100_c = RL_default(h,g,100,mode='crop')
        rlc_250_c = RL_default(h,g,250,mode='crop')
        # plot
        plot_images([
                h, rlc_5_c, rlc_100_c, rlc_250_c
            ],[
                r'$h$', r'$f_{RL~crop~5}$', r'$f_{RL~crop~100}$', r'$f_{RL~crop~250}$'
            ])
    else:
        random_mask = noise_level * np.random.randn(h.shape[0],h.shape[1])
        h_noise = h + random_mask
        h_noise = normalize_image(h_noise)
        rlc_5_c = RL_default(h_noise,g,5,mode='crop')
        rlc_100_c = RL_default(h_noise,g,100,mode='crop')
        rlc_250_c = RL_default(h_noise,g,250,mode='crop')
        # plot
        plot_images([
                h_noise, rlc_5_c, rlc_100_c, rlc_250_c
            ],[
                r'$h_{noise}$', r'$f_{RL~crop~5}$', r'$f_{RL~crop~100}$', r'$f_{RL~crop~250}$'
            ])
    # print similarity mertics
    f_crop = get_kernel(f,g,mode='crop')
    print_quant('cropped RL at 5 iterations',rlc_5_c,f_crop)
    print_quant('cropped RL at 100 iterations',rlc_100_c,f_crop)
    print_quant('cropped RL at 250 iterations',rlc_250_c,f_crop)

rl_crop_gallery(noise=False)

In [ ]:
# PLOT HIGHLIGHTS (CHOOSE ONE): CROPPED RICHARDSON-LUCY (NO NOISE, BLUR)
#----------------------------------------------------------------
# uncomment only the selection which you want to see
#single(rlc_5_c, 100, 0, 0)
#single(rlc_100_c, 100, 0, 0)
#single(rlc_250_c, 100, 0, 0)

In [ ]:
# SEMI-BLIND DECONVOLUTION: CROPPED RL --> GRAPH
#----------------------------------------------------------------
def graph_rl_crop(params=250):
    # create a list to store the results in
    result = RL_default(h,g,params,mode='crop',quants=True)[1]
    # call the plot function for the list of parameters and the quant array
    plot_quants(np.arange(0, params, 1), result, xl='Iterations')

graph_rl_crop()

Combining the crop with a subsequent reflective padding even further improves the result.

In [ ]:
# SEMI-BLIND DECONVOLUTION: CROPPED REFLECTIVE PADDING
#----------------------------------------------------------------
# produce a cropped reflected example of the blurred image
h_crop = get_kernel(h,g,mode='crop')
h_crop_refl = np.pad(h_crop, int(np.ceil(h.shape[0]*0.1)), mode='reflect')

def rl_crop_refl_gallery(noise=False, noise_level=0.1):
    # calculate the reflected custom RL deconvolutions at different numbers of iterations
    if not noise:
        h_crop = get_kernel(h,g,mode='crop')
        rlc_5_cr = RL_default(h_crop,g,5,mode='reflect')
        rlc_100_cr = RL_default(h_crop,g,100,mode='reflect')
        rlc_250_cr = RL_default(h_crop,g,250,mode='reflect')
        # plot
        plot_images([
                h_crop_refl, rlc_5_cr, rlc_100_cr, rlc_250_cr
            ],[
                r'$h_{crop+reflect}$', r'$RL_{crop+reflect~5}$', r'$RL_{crop+reflect~100}$', r'$RL_{crop+reflect~250}$'
            ])
    else:
        random_mask = noise_level * np.random.randn(h.shape[0],h.shape[1])
        h_noise = h + random_mask
        h_noise = normalize_image(h_noise)
        h_crop = get_kernel(h_noise,g,mode='crop')
        h_crop_refl = np.pad(h_crop, int(np.ceil(h.shape[0]*0.1)), mode='reflect')
        rlc_5_cr = RL_default(h_crop,g,5,mode='reflect')
        rlc_100_cr = RL_default(h_crop,g,100,mode='reflect')
        rlc_250_cr = RL_default(h_crop,g,250,mode='reflect')
        # plot
        plot_images([
                h_crop_refl, rlc_5_cr, rlc_100_cr, rlc_250_cr
            ],[
                r'$h_{crop+reflect noise}$', r'$RL_{crop+reflect~5}$', r'$RL_{crop+reflect~100}$', r'$RL_{crop+reflect~250}$'
            ])
    # print similarity mertics
    # NOTE: to perform this comparison both images need to be of the same dimensions, so we need to crop the ground-truth as well
    f_crop = get_kernel(f,g,mode='crop')
    print_quant('cropped reflective RL at 5 iterations',rlc_5_cr,f_crop)
    print_quant('cropped reflective RL at 100 iterations',rlc_100_cr,f_crop)
    print_quant('cropped reflective RL at 250 iterations',rlc_250_cr,f_crop)

rl_crop_refl_gallery(noise=True)

In [ ]:
# PLOT HIGHLIGHTS (CHOOSE ONE): CROPPED REFLECTIVE RICHARDSON-LUCY (NO NOISE, BLUR)
#----------------------------------------------------------------
# uncomment only the selection which you want to see
#single(rlc_5_cr, 100, 0, 0)
#single(rlc_100_cr, 100, 0, 0)
#single(rlc_250_cr, 100, 0, 0)

In [ ]:
# PLOT HIGHLIGHTS (CHOOSE ONE): EXTREMELY HIGH NUMBER OF ITERATIONS
# NOTE: high numbers of iterations significantly increase performance requirements
#----------------------------------------------------------------
# uncomment whichever number of RL deconvolution iteration steps you want to perform
#rlc_1000_cr = RL_default(h_crop,g,1000,mode='reflect')
#rlc_2500_cr = RL_default(h_crop,g,2500,mode='reflect')
#rlc_var_cr = RL_default(h_crop,g,X,mode='reflect') # replace 'X' with the desired number of iterations
# uncomment only the selection which you want to see
#single(rlc_1000_cr, 100)
#single(rlc_2500_cr, 100)
#single(rlc_var_cr, 100)

In [ ]:
# SEMI-BLIND DECONVOLUTION: CROPPED REFELCTIVE RL --> GRAPH
#----------------------------------------------------------------
def graph_rl_cr(params=250):
    # create a list to store the results in
    result = RL_default(h,g,params,mode='crofl',quants=True)[1]
    # call the plot function for the list of parameters and the quant array
    plot_quants(np.arange(0, params, 1), result, xl='Iterations')

graph_rl_cr()

Now for the iterative Richardson-Lucy method, including the previously mentioned improvements. Clipping the pixel values at a maximum to prevent artificial darkening helps with visual clarity in the following examples.

In [ ]:
# BLIND DECONVOLUTION: ITERATIVE RICHARDSON-LUCY (iRL), CROPPED REFLECTIVE PADDING WITH CLIPPING
#----------------------------------------------------------------
def irl_gallery(noise=False, noise_level=0.1):
    # define constants
    base_original = h
    psf = matlab_style_gauss2D(shape=(15,15), sigma=2)
    if not noise:
        # compute both the clipped and non-clipped iRL deconvolution results
        irl_blind_clip_3_5 = get_kernel(RL_iter(h_crop_refl,psf,f,3,5,clip_f=True)[0],base_original)
        irl_blind_clip_3_15 = get_kernel(RL_iter(h_crop_refl,psf,f,3,15,clip_f=True)[0],base_original)
        irl_blind_noclip_3_5 = get_kernel(RL_iter(h_crop_refl,psf,f,3,5,clip_f=False)[0],base_original)
        irl_blind_noclip_3_15 = get_kernel(RL_iter(h_crop_refl,psf,f,3,15,clip_f=False)[0],base_original)
        # plot
        plot_images([
                irl_blind_clip_3_5, irl_blind_clip_3_15, irl_blind_noclip_3_5, irl_blind_noclip_3_15
            ],[
                r'$i=3,k=5, clip$', r'$i=3,k=15, clip$', r'$i=3,k=5, noclip$', r'$i=3,k=15, noclip$'
            ])
    else:
        random_mask = noise_level * np.random.randn(h.shape[0],h.shape[1])
        h_noise = h + random_mask
        h_noise = normalize_image(h_noise)
        h_crop = get_kernel(h_noise,g,mode='crop')
        h_crop_refl = np.pad(h_crop, int(np.ceil(h.shape[0]*0.1)), mode='reflect')
        # compute both the clipped and non-clipped iRL deconvolution results
        irl_blind_clip_3_5 = get_kernel(RL_iter(h_crop_refl,psf,f,3,5,clip_f=True)[0],base_original)
        irl_blind_clip_3_15 = get_kernel(RL_iter(h_crop_refl,psf,f,3,15,clip_f=True)[0],base_original)
        irl_blind_noclip_3_5 = get_kernel(RL_iter(h_crop_refl,psf,f,3,5,clip_f=False)[0],base_original)
        irl_blind_noclip_3_15 = get_kernel(RL_iter(h_crop_refl,psf,f,3,15,clip_f=False)[0],base_original)
        # plot
        plot_images([
                irl_blind_clip_3_5, irl_blind_clip_3_15, irl_blind_noclip_3_5, irl_blind_noclip_3_15
            ],[
                r'$i=3,k=5, clip~noise$', r'$i=3,k=15, clip~noise $', r'$i=3,k=5, noclip~noise$', r'$i=3,k=15, noclip~noise$'
            ])
    # print similarity mertics
    # NOTE: to perform this comparison both images need to be of the same dimensions, so we need to crop the ground-truth as well
    f_crop = get_kernel(f,g,mode='crop')
    print_quant('cropped reflective iRL at 3 internal and 5 total iterations (clip)',irl_blind_clip_3_5,f)
    print_quant('cropped reflective iRL at 3 internal and 15 total iterations (clip)',irl_blind_clip_3_15,f)
    print_quant('cropped reflective iRL at 3 internal and 5 total iterations (no clip)',irl_blind_noclip_3_5,f)
    print_quant('cropped reflective iRL at 3 internal and 15 total iterations (no clip)',irl_blind_noclip_3_15,f)

irl_gallery(noise=True)

In [ ]:
# BLIND DECONVOLUTION: CROPPED REFELCTIVE iRL --> GRAPH (no noise)
#----------------------------------------------------------------
def graph_irl_cr(params=250):
    random_mask = noise_level * np.random.randn(h.shape[0],h.shape[1])
    h_noise = h + random_mask
    h_noise = normalize_image(h_noise)
    h_crop = get_kernel(h_noise,g,mode='crop')
    h_crop_refl = np.pad(h_crop, int(np.ceil(h.shape[0]*0.1)), mode='reflect')
    # create a list to store the results in
    result = RL_iter(h_crop_refl,psf,f,3,params,clip_f=True,quants=True)[2]
    # call the plot function for the list of parameters and the quant array
    plot_quants(np.arange(0, params, 1), result, xl='Iterations', legloc=6)

graph_irl_cr()

In [ ]:
""" random_mask = 0.03 * np.random.randn(h.shape[0],h.shape[1])
h_noise = h + random_mask
h_noise = normalize_image(h_noise)
h_crop = get_kernel(h_noise,g,mode='crop')
h_crop_refl = np.pad(h_crop, int(np.ceil(h.shape[0]*0.1)), mode='reflect')
ok = RL_iter(h_crop_refl,psf,f,3,70,clip_f=True,clip_g=True)
single(ok[1]) """

In [ ]:
#single(get_kernel(ok[0],base_original))

In [ ]:
""" # BLIND DECONVOLUTION: ITERATIVE RICHARDSON-LUCY (iRL), CROPPED REFLECTIVE PADDING WITH CLIPPING
#----------------------------------------------------------------

# define constants
base_original = h_noise
h_c_noise = get_kernel(h_noise,g,mode='crop')
h_cr_noise = np.pad(h_c_noise, int(np.ceil(h.shape[0]*0.1)), mode='reflect')
psf = matlab_style_gauss2D(shape=(15,15), sigma=2)
# compute both the clipped and non-clipped iRL deconvolution results
irl_cr_noise_3_5 = get_kernel(RL_iter(h_cr_noise,psf,f,3,5,clip_f=True)[0],base_original)
irl_cr_noise_3_15 = get_kernel(RL_iter(h_cr_noise,psf,f,3,15,clip_f=True)[0],base_original)
irl_cr_noise_3_5_nc = get_kernel(RL_iter(h_cr_noise,psf,f,3,5,clip_f=False)[0],base_original)
irl_cr_noise_3_15_nc = get_kernel(RL_iter(h_cr_noise,psf,f,3,15,clip_f=False)[0],base_original)
# plot
plot_images([
        irl_cr_noise_3_5, irl_cr_noise_3_15, irl_cr_noise_3_5_nc, irl_cr_noise_3_15_nc
    ],[
        r'$i=3,k=5, clip$', r'$i=3,k=15, clip$', r'$i=3,k=5, noclip$', r'$i=3,k=15, noclip$'
    ])
# print similarity mertics
# NOTE: to perform this comparison both images need to be of the same dimensions, so we need to crop the ground-truth as well
f_crop = get_kernel(f,g,mode='crop')
print_quant('cropped reflective iRL at 3 internal and 5 total iterations (clip)',irl_cr_noise_3_5,f)
print_quant('cropped reflective iRL at 3 internal and 15 total iterations (clip)',irl_cr_noise_3_15,f)
print_quant('cropped reflective iRL at 3 internal and 5 total iterations (no clip)',irl_cr_noise_3_5_nc,f)
print_quant('cropped reflective iRL at 3 internal and 15 total iterations (no clip)',irl_cr_noise_3_15_nc,f) """

In [ ]:
# BLIND DECONVOLUTION: CROPPED REFELCTIVE iRL --> GRAPH (noise)
#----------------------------------------------------------------
def graph_irl_cr_noise(params=150):
    h_c_noise = get_kernel(h_noise,g,mode='crop')
    h_cr_noise = np.pad(h_c_noise, int(np.ceil(h.shape[0]*0.1)), mode='reflect')
    # create a list to store the results in
    result = RL_iter(h_cr_noise,psf,f,3,params,clip_f=True,quants=True)[2]
    # call the plot function for the list of parameters and the quant array
    plot_quants(np.arange(0, params, 1), result, xl='Iterations', legloc=6)

graph_irl_cr_noise()

### Machine Learning Methods

##### Loading data

In [ ]:
# LOAD DATA (RUN ONCE)
#----------------------------------------------------------------
def load_data(dataset='mnist'):
    if dataset == 'mnist':
        # load data from mnist dataset
        (train, _), (test, _) = mnist.load_data()
        # we should also scale/normalize our data
        train = train.reshape([-1,28,28,1]) / 255
        test = test.reshape([-1,28,28,1]) / 255
        #print(f'train shape = {train.shape}, test shape = {test.shape}')
        return train, test
    elif dataset == 'fashionmnist':
        (train_f, _), (test_f, _) = fashion_mnist.load_data()
        (train_m, _), (test_m, _) = mnist.load_data()
        # we should also scale/normalize our data
        train_f = train_f.reshape([-1,28,28,1]) / 255
        test_f = test_f.reshape([-1,28,28,1]) / 255
        train_m = train_m.reshape([-1,28,28,1]) / 255
        test_m = test_m.reshape([-1,28,28,1]) / 255
        # now we can concatenate our sets
        train = np.concatenate([train_f,train_m])
        test = np.concatenate([test_f,test_m])
        #print(f'train shape = {train.shape}, test shape = {test.shape}')
        return train, test
    elif dataset == 'cifar10':
        (train, _), (test, _) = cifar10.load_data()
        # we should also scale/normalize our data
        train = np.asarray([cv2.cvtColor(i, cv2.COLOR_BGR2GRAY) for i in train])
        test = np.asarray([cv2.cvtColor(i, cv2.COLOR_BGR2GRAY) for i in test])
        train = train.reshape([-1,32,32,1]) / 255
        test = test.reshape([-1,32,32,1]) / 255
        #print(f'train shape = {train.shape}, test shape = {test.shape}')
        return train, test
    elif dataset == 'lfw':
        # NOTE: image selection HEAVILY impacts performance, select only as many images as your hardware allows
        images = np.asarray([cv2.imread(file, 0) for file in glob.glob("./data/lfw/[a-c]*/*")])
        images = np.asarray([cv2.resize(i, (256, 256)) for i in images])
        split_percentage = math.ceil(len(images)*0.8)
        train = images[:split_percentage]
        test = images[split_percentage:]
        #train = np.asarray([tf.expand_dims(i,2) for i in train]) / 255
        train = train.reshape([-1,256,256,1]) / 255
        test = test.reshape([-1,256,256,1]) / 255
        #print(f'train shape = {train.shape}, test shape = {test.shape}')
        return train, test
    

train, test = load_data(dataset='mnist')

# with the following we display the dimensions of the split dataset
print(f'The training subset has size: {train.shape[0]}, and the test subset has size: {test.shape[0]}.')
print(f'Each of the images in these subsets have dimensions: {train.shape[1:]}.')
print(f'The range of values in each of the images/arrays in these subsets is: [{train[0].min()}, {train[0].max()}].')

In [ ]:
# SAMPLE CLEAN IMAGES
#----------------------------------------------------------------
sample_train_clean = []
rows = 2 # """ variable (default=3) """
sample_train_list = range(4*rows)
for i in sample_train_list:
    sample_train_clean.append(train[i])
plot_images(sample_train_clean, [f'train[{x}]' for x in sample_train_list], colormap='OrRd')

##### Create and sample blurred images

In [ ]:
# ADD BLUR (RUN ONCE)
#----------------------------------------------------------------
# we can create a simple blur kernel, normalize it and match the dimensions to the dataset
# note that generally odd integer dimensions are preferred
blur_kernel = np.ones(shape=(5,5)) # """ variable (default=np.ones(shape=(5,5))) """
blur_kernel /= np.sum(blur_kernel)
blur_kernel = np.atleast_3d(blur_kernel)
# now we can create two empty arrays to be filled with our blurred images
train_blur = [fftc(train[i], blur_kernel, 'same') for i in range(len(train))]
test_blur = [fftc(test[i], blur_kernel, 'same') for i in range(len(test))]

train_blur = np.asarray(train_blur)
test_blur = np.asarray(test_blur)

print(f'Blurred copy of dataset has been created.')

In [ ]:
# SAMPLE BLURRED IMAGES
#----------------------------------------------------------------
sample_train_blur = []
sample_train_blur_number = 4 # """ variable (default=4) """
for i in range(sample_train_blur_number):
    sample_train_blur.append(train[i])
    sample_train_blur.append(train_blur[i])

plot_images(sample_train_blur,['train','blur']*sample_train_blur_number, colormap='OrRd')

##### Create and sample noisy images

In [ ]:
# ADD NOISE (RUN ONCE)
#----------------------------------------------------------------
# we can define the level of random noise we want in our images and apply it to each of our images
noise = 0.3 # """ variable (default=0.3) """
train_noise = train + noise * np.random.normal(0, 1, size=train.shape)
test_noise = test + noise * np.random.normal(0, 1, size=test.shape)

train_noise = np.asarray(train_noise)
test_noise = np.asarray(test_noise)

print(f'Noisy copy of dataset has been created.')

In [ ]:
# SAMPLE NOISY IMAGES
#----------------------------------------------------------------
sample_train_noise = []
sample_train_noise_number = 4 # """ variable (default=4) """
for i in range(sample_train_noise_number):
    sample_train_noise.append(train[i])
    sample_train_noise.append(train_noise[i])

plot_images(sample_train_noise,['train','noise']*sample_train_noise_number, colormap='OrRd')

##### Create and sample noisy blurred images

In [ ]:
# ADD NOISE TO BLUR (RUN ONCE)
#----------------------------------------------------------------
# we can define the level of random noise we want in our images and apply it to each of our images
noise = 0.03 # """ variable (default=0.3) """
train_blur_noise = train_blur + noise * np.random.normal(0, 1, size=train.shape)
test_blur_noise = test_blur + noise * np.random.normal(0, 1, size=test.shape)

train_blur_noise = np.asarray(train_blur_noise)
test_blur_noise = np.asarray(test_blur_noise)

print(f'Noisy copy of blurred dataset has been created.')

In [ ]:
# SAMPLE NOISY BLURRED IMAGES
#----------------------------------------------------------------
sample_train_blur_noise = []
sample_train_blur_noise_number = 4 # """ variable (default=4) """
for i in range(sample_train_blur_noise_number):
    sample_train_blur_noise.append(train[i])
    sample_train_blur_noise.append(train_blur_noise[i])

plot_images(sample_train_blur_noise,['train','blur+noise']*sample_train_blur_noise_number, colormap='OrRd')

##### Construct Machine Learning models and summarize

In [ ]:
# DEFINING OUR MODELS AND DEFINE LATEST MODEL TRAINING SET
#----------------------------------------------------------------
def model_simple():
    mod = Sequential([
        Input(shape=(None,None,1), name='input'),
        # then we apply a 2D convolution with a ReLu activation applied to its output
        Conv2D(32, 3, activation='relu', padding='same', name='Conv2D-0'),
        # then we apply a max-pooling, to decrease the size of our intermediate layers and speed up the learning process
        MaxPool2D(name='MaxPool2D-0'),
        # as a regularization layer we use a simple dropout instead of an L2 regularization to prevent overfitting
        #Dropout(0.3, name='Dropout-0'),
        # the following layers are the same as before
        Conv2D(32, 3, activation='relu', padding='same', name='Conv2D-1'),
        MaxPool2D(name='MaxPool2D-1'),
        Conv2D(32, 3, activation='relu', padding='same', name='Conv2D-2'),
        # to get back a representation with the same dimensions we need to upsample
        UpSampling2D(name='UpSampling2D-0'),
        #Dropout(0.3, name='Dropout-1'),
        Conv2D(32, 3, activation='relu', padding='same', name='Conv2D-3'),
        UpSampling2D(name='UpSampling2D-1'),
        # finally we can use a sigmoid activation 2D convolution layer to output our estimated de-noised and de-blurred image
        Conv2D(1, 3, activation='sigmoid', padding='same', name='Conv2D-4')
    ])

    return mod, 'simple'

def skip_model():
    inputs = Input(shape=(None, None, 1))
    # encoder
    conv1 = Conv2D(32, 3, 1, padding='same')(inputs)
    conv2 = Conv2D(64, 3, 1, activation = 'relu', padding = 'same')(conv1)
    conv3 = Conv2D(128, 3, 1, activation = 'relu', padding = 'same')(conv2)
    # decoder
    up1 = Conv2D(128, 3, activation = 'relu', padding = 'same')(UpSampling2D(size = 1)(conv3))
    # now we use a skip connection to return `merge1` as being `inputs + conv1`
    skip1 = Add()([conv3,up1])
    up2 = Conv2D(64, 3, activation = 'relu', padding = 'same')(UpSampling2D(size = 1)(skip1))
    skip2 = Add()([conv2,up2])
    up3 = Conv2D(32, 3, activation = 'relu', padding = 'same')(UpSampling2D(size = 1)(skip2))
    skip3 = Add()([conv1,up3])
    # lastly we run a convolution to return a result in the correct shape
    norm_conv = Conv2D(1, 3, activation='sigmoid', padding='same')(up3)

    mod = Model(inputs=inputs, outputs=norm_conv)
    
    return mod, 'skip'

model, model_name = model_simple()

model.compile(
    # the adam optimizer is the standard and seems to be working well
    # feel free to change to taste and experiment
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

latest_model = 'none'

print(f'Model "{model_name}" has been selected, and latest training set used model is {latest_model}.')

In [ ]:
# SUMMARIZE CHOSEN MODEL
#----------------------------------------------------------------
model.summary()

##### Define a 'curriculum' for the model

In [ ]:
# DEFINE TRAINING FUNCTION
# When using logging (log=True) a TensorBoard log will be written to the 'logs' folder.
#   Using the 'opentensorboard.bat' file will automatically open from the correct folder,
#   Changing the file structure requires running a custom TensorBoard command to open the software and read the logs from the correct directory.
# --------------------------------------------------------------
def train_model(name='blur_noise', log=False, train_in=train_blur_noise, train_target=train, test_in=test_blur_noise, test_target=test, epochs=10, batch_size=128):
    """ Train model on training data. 
    name = name of the model for logging purposes
    log = boolean indicating if the model should log to file
    train_in = the training data of the perturbed images
    train_target = the corresponding target data of clean images
    test_in = the test data of the perturbed images
    test_target = the corresponding target data of clean images
    epochs = the number of epochs to run model for
    batch_size = the number of images in a batch (adjust as needed for dataset) """

    if log:
        # initialize model tensorboard log
        log_name = f'{name}_{int(time.time())}'
        tensorboard = TensorBoard(log_dir=f'logs/{log_name}')

        # train model on noisy blurred images
        history = model.fit(
                        train_in,
                        train_target,
                        epochs=epochs,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_data=(test_in, test_target),
                        callbacks=[tensorboard]
                    )
    else:
        # train model on noisy blurred images
        history = model.fit(
                        train_in,
                        train_target,
                        epochs=epochs,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_data=(test_in, test_target)
                    )

    print(f'Model has been trained. \n Please reconsider before running this function again, as it can prove exceedingly resource exhaustive. ')

##### Train model and run predictions (noise+blur)

In [ ]:
# START TRAINING SELECTED MODEL (noise+blur)
#----------------------------------------------------------------
if latest_model != 'blur_noise':
    train_model()
    latest_model = 'blur_noise'

In [ ]:
# RUN PREDICTION AND SAMPLE RESULTS (noise+blur)
#----------------------------------------------------------------
def sample_model(set_mod, set_clean, model, num_imgs=2, random=True):
    # set the index of the sampled images
    index = 0
    if random:
        index = np.random.randint(0, len(set_mod))

    # select the modified/perturbed images
    test_images = set_mod[index:index+num_imgs]
    # perform a prediction on these images
    test_results = model.predict(test_images)
    # select trained images or the ground truth
    train_images = set_clean[index:index+num_imgs]

    sample_predictions = []
    for i in range(num_imgs):
        sample_predictions.append(test_images[i])
        sample_predictions.append(test_results[i])
        sample_predictions.append(train_images[i])

    plot_images(sample_predictions, ['test','result','train']*num_imgs, n_split=3, colormap='gray')

sample_model(test_blur_noise, test, model)

##### Apply model to image of Col. Collins
Both the strict input dimension method and the general, size-ambiguous methods

In [ ]:
# READ IMAGE OF COL. EILEEN COLLINS INTO MEMORY AND APPLY MODIFICATIONS
#----------------------------------------------------------------
eileen_base = rgb2gray(data.astronaut()).astype('float32')
psf = np.ones((5,5))
psf /= np.sum(psf)

eileen_blur = conv_prod(eileen_base,psf)

# If we add a generic distortion in the form of random noise we get:
random_mask = 0.1 * np.random.randn(eileen_base.shape[0],eileen_base.shape[1])
f_noise = eileen_base + random_mask
eileen_noise = normalize_image(f_noise)
h_noise = eileen_blur + random_mask
eileen_blur_noise = normalize_image(h_noise)

In [ ]:
# APPLY PREDICTION TO FULL IMAGE (28pixel method)
#----------------------------------------------------------------
# resize from a 512x512 image to 28x28 (add dummy dimensions to fit the input shape)
eileen_blur_noise_reshape = cv2.resize(eileen_blur_noise, (28, 28))
eileen_blur_noise_reshape = np.expand_dims(eileen_blur_noise_reshape, axis=2)
eileen_blur_noise_reshape = np.expand_dims(eileen_blur_noise_reshape, axis=0)

# finally we can apply our model to the blurred, noisy and significantly resized image
eileen_result = model.predict(eileen_blur_noise_reshape)

plot_images([
        eileen_blur_noise, eileen_blur_noise_reshape[0], eileen_result[0]
    ],[
        f'blurred+noise', f'reshaped', f'prediction'
    ], colormap='gray', n_split=3)

In [ ]:
# CUTTING UP EILEEN
#----------------------------------------------------------------
eileen_chunks = into_chunks(eileen_blur_noise)
# optionally plot
#plot_images(eileen_chunks[0], n_split=eileen_chunks[1], colormap='gray', make_title=False)

In [ ]:
# APPLYING MODEL TO CHUNKS
#----------------------------------------------------------------
eileen_chunks_reshaped = []
for i in range(eileen_chunks[0].shape[0]):
    reshape_0 = cv2.resize(eileen_chunks[0][i], (28, 28))
    reshape_1 = np.expand_dims(eileen_chunks[0][i], axis=2)
    eileen_chunks_reshaped.append(reshape_1)
eileen_chunks_reshaped = np.asarray(eileen_chunks_reshaped)

eileen_chunks_prediction = model.predict(eileen_chunks_reshaped, batch_size=32)
plot_images(eileen_chunks_prediction, n_split=eileen_chunks[1], colormap='gray', make_title=False)

In [ ]:
# APPLY PREDICTION TO FULL IMAGE (full method)
# we can apply our model to the blurred noisy image
eileen_result = model.predict(batchify(eileen_blur_noise))
# define the iRL deconvolution of the prediction result
eileen_c_blur_noise_result = get_kernel(eileen_result[0,:,:,0],psf,mode='crop')
eileen_cr_blur_noise_result = np.pad(eileen_c_blur_noise_result, int(np.ceil(eileen_base.shape[0]*0.1)), mode='reflect')
# compute both the clipped and non-clipped iRL deconvolution results
ml_irl_cr_blur_noise_result_3_15 = get_kernel(RL_iter(eileen_cr_blur_noise_result,psf,eileen_base,3,15,clip_f=True)[0],eileen_base)
# print similarity mertics
print_quant('ML denoised image',eileen_result[0,:,:,0],eileen_base)
print_quant('iRL with i=3,k=15, crop+refl. of ML denoised image',ml_irl_cr_blur_noise_result_3_15,eileen_base)
# plot
plot_images([
        eileen_blur_noise, eileen_result[0], ml_irl_cr_blur_noise_result_3_15
    ],[
        f'blur+noise', f'prediction', f'deconv'
    ], colormap='gray', n_split=3)

##### Train model and run predictions (noise)

In [ ]:
# START TRAINING SELECTED MODEL (noise)
#----------------------------------------------------------------
if latest_model != 'noise':
    train_model(name='noise', log=False, train_in=train_noise, train_target=train, test_in=test_noise, test_target=test)
    latest_model = 'noise'

In [ ]:
# APPLY PREDICTION TO FULL IMAGE (NOISE ONLY)
#----------------------------------------------------------------
# we can apply our model to the noisy image
eileen_result = model.predict(batchify(eileen_noise))
# define the iRL deconvolution of the prediction result
eileen_c_noise_result = get_kernel(eileen_result[0,:,:,0],psf,mode='crop')
eileen_cr_noise_result = np.pad(eileen_c_noise_result, int(np.ceil(eileen_base.shape[0]*0.1)), mode='reflect')
# compute both the clipped and non-clipped iRL deconvolution results
ml_irl_cr_noise_result_3_15 = get_kernel(RL_iter(eileen_cr_noise_result,psf,eileen_base,3,15,clip_f=True)[0],eileen_base)
# print similarity mertics
print_quant('ML denoised image',eileen_result[0,:,:,0],eileen_base)
print_quant('iRL with i=3,k=15, crop+refl. of ML denoised image',ml_irl_cr_noise_result_3_15,eileen_base)
# plot
plot_images([
        eileen_noise, eileen_result[0], ml_irl_cr_noise_result_3_15
    ],[
        f'noise', f'prediction', f'deconv'
    ], colormap='gray', n_split=3)

##### Train model and run predictions (blur)

In [ ]:
# START TRAINING SELECTED MODEL (blur)
#----------------------------------------------------------------
if latest_model != 'blur':
    train_model(name='blur', log=False, train_in=train_blur, train_target=train, test_in=test_blur, test_target=test)
    latest_model = 'blur'

In [ ]:
# APPLY PREDICTION TO FULL IMAGE (BLUR ONLY)
#----------------------------------------------------------------
# we can apply our model to the blurred image
eileen_result = model.predict(batchify(eileen_blur))
# define the iRL deconvolution of the prediction result
eileen_c_blur_result = get_kernel(eileen_result[0,:,:,0],psf,mode='crop')
eileen_cr_blur_result = np.pad(eileen_c_blur_result, int(np.ceil(eileen_base.shape[0]*0.1)), mode='reflect')
# compute both the clipped and non-clipped iRL deconvolution results
ml_irl_cr_blur_result_3_15 = get_kernel(RL_iter(eileen_cr_blur_result,psf,eileen_base,3,15,clip_f=True)[0],eileen_base)
# print similarity mertics
print_quant('ML denoised image',eileen_result[0,:,:,0],eileen_base)
print_quant('iRL with i=3,k=15, crop+refl. of ML denoised image',ml_irl_cr_blur_result_3_15,eileen_base)
# plot
plot_images([
        eileen_blur, eileen_result[0], ml_irl_cr_blur_result_3_15
    ],[
        f'blur', f'prediction', f'deconv'
    ], colormap='gray', n_split=3)